In [1]:
from roler.model import ModelPrior
from roler.distributions import *

prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

In [2]:
from roler.simulation import Simulator

simulator = Simulator()
theta = prior.sample()
stats = simulator.simulate(theta)
stats

R[write to console]: Using GitHub PAT from the git credential store.



Installing the 'roleR' R package from GitHub...


R[write to console]: Skipping install of 'roleR' from a github remote, the SHA1 (cc6546a1) has not changed since last install.
  Use `force = TRUE` to force installation



,hill_abund_1,hill_abund_2,hill_abund_3,hill_abund_4,hill_gen_1,hill_gen_2,hill_gen_3,hill_gen_4,hill_trait_1,hill_trait_2,hill_trait_3,hill_trait_4,hill_phy_1,hill_phy_2,hill_phy_3,hill_phy_4,richness,iteration
1,1.000000,1.000000,1.000000,1.000000,1.0,inf,inf,inf,NaN,NaN,NaN,NaN,1,2,3,4,1,0.0
2,1.048688,1.016528,1.012422,1.011034,1.0,inf,inf,inf,11.090909,11.090909,11.090909,11.090909,1,2,3,4,2,10.0
3,1.099673,1.033468,1.025103,1.022284,1.0,inf,inf,inf,11.359543,11.182945,11.160154,11.152440,1,2,3,4,3,20.0
4,1.099673,1.033468,1.025103,1.022284,1.0,inf,inf,inf,11.359543,11.182945,11.160154,11.152440,1,2,3,4,3,30.0
5,1.195290,1.068485,1.051273,1.045451,1.0,inf,inf,inf,10.029241,9.461527,9.161846,8.930236,1,2,3,4,4,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,17.181017,12.808950,10.853102,9.806134,1.0,inf,inf,inf,20.830403,15.766855,13.383629,12.041571,1,2,3,4,27,1960.0
198,17.096891,13.079086,11.212333,10.193850,1.0,inf,inf,inf,20.772194,15.974402,13.577508,12.182532,1,2,3,4,26,1970.0
199,16.625485,12.965157,11.240384,10.286193,1.0,inf,inf,inf,21.035919,16.524323,14.215648,12.863856,1,2,3,4,25,1980.0
200,16.959463,13.171681,11.400963,10.405858,1.0,inf,inf,inf,21.434094,16.513922,14.105285,12.736800,1,2,3,4,26,1990.0


In [5]:
from roler.datasets import Dataset

dataset = Dataset(simulator=simulator, prior=prior)
theta, x = dataset.generate_dataset(samples=10)

print("theta.size", theta.shape)
print("x.size", x.shape)

theta.size torch.Size([1995, 2])
x.size torch.Size([1995, 9])


In [6]:
from sbi.inference import SNPE

snpe = SNPE(prior=prior.get_joint_uniform())
density_estimator = snpe.append_simulations(theta, x).train()
posterior = snpe.build_posterior(density_estimator)

/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Neural network successfully converged after 289 epochs.

In [15]:
theta_true, x_obs = dataset.generate_dataset(samples=1)
theta_true, x_obs = theta_true[-1], x_obs[-1]
print("Observed simulation output:", x_obs)

# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = posterior.sample((1000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)

Observed simulation output: tensor([23.7825, 17.4744, 15.0638, 13.9038, 25.6987, 18.4144, 15.5630, 14.2175,
        40.0000])


Drawing 1000 posterior samples: 1079it [00:00, 209666.62it/s]           

Posterior samples shape: torch.Size([1000, 2])
Posterior mean estimate: tensor([164.5733, 747.5741])

Theta True      : tensor([221.4486, 467.5372])
Theta Prediction: tensor([164.5733, 747.5741])


In [23]:
print("True       :", dataset.get_params_from_tensor(theta_true))
print("Prediction :", dataset.get_params_from_tensor(posterior_mean))

True       : ModelParams(individuals_local=221, individuals_meta=468, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
Prediction : ModelParams(individuals_local=165, individuals_meta=748, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
